<a href="https://colab.research.google.com/github/purvasingh96/Deep-Reinforcement-Learning/blob/master/4.%20Deep%20Q%20Networks/Credit_Card_Fraud_Detection_via_DQNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading data and Making *gym-fraud-detection* environment

In [0]:
import shutil

shutil.rmtree('/content/gym-fraud-detection')

In [3]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/creditcard.csv  


In [4]:
import os
print(os.getcwd())

/content


In [5]:
!unzip gym-fraud-detection.zip

Archive:  gym-fraud-detection.zip
   creating: gym-fraud-detection/
  inflating: gym-fraud-detection/setup.py  
  inflating: gym-fraud-detection/README.md  
   creating: gym-fraud-detection/gym_fraud_detection/
  inflating: gym-fraud-detection/gym_fraud_detection/__init__.py  
   creating: gym-fraud-detection/.idea/
  inflating: gym-fraud-detection/.idea/workspace.xml  
  inflating: gym-fraud-detection/.idea/vcs.xml  
  inflating: gym-fraud-detection/.idea/gym-fraud-detection.iml  
  inflating: gym-fraud-detection/.idea/modules.xml  
  inflating: gym-fraud-detection/.idea/encodings.xml  
  inflating: gym-fraud-detection/.idea/misc.xml  
   creating: gym-fraud-detection/.git/
  inflating: gym-fraud-detection/.git/index  
  inflating: gym-fraud-detection/.git/FETCH_HEAD  
 extracting: gym-fraud-detection/.git/ORIG_HEAD  
 extracting: gym-fraud-detection/.git/COMMIT_EDITMSG  
 extracting: gym-fraud-detection/.git/HEAD  
  inflating: gym-fraud-detection/.git/packed-refs  
  inflating: gym-

In [0]:
import os
os.chdir('gym-fraud-detection')

In [7]:
pip install -e .

Obtaining file:///content/gym-fraud-detection
  Running setup.py develop for gym-fraud-detection


In [0]:
import gym
import gym_fraud_detection

In [0]:
os.chdir('/content')
env = gym.make('gym-fraud-detection-v0')

# Download proper libraries

In [0]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


In [0]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exploring the custom gym environment

In [13]:
print(env.action_space.n)

2


In [0]:
actions = []
rewards = []
count = 5

while True:
  action = env.action_space.sample()
  reward = env.step(action)
  actions.append(action)
  rewards.append(reward)
  count -= 1
  if count==0:
    break


In [15]:
for i in range(5):
  print("action : ", actions[i], "labelled data : ", env.label_for(i))
  print("reward : ", rewards[i])

action :  0 labelled data :  0.0
reward :  (1, 1, False, '{"true_positive_rate": 0, "false_positive_rate": 0, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  1 labelled data :  0.0
reward :  (2, -1, False, '{"true_positive_rate": 0, "false_positive_rate": 1, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  0 labelled data :  0.0
reward :  (3, 1, False, '{"true_positive_rate": 0, "false_positive_rate": 1, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  1 labelled data :  0.0
reward :  (4, -1, False, '{"true_positive_rate": 0, "false_positive_rate": 2, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  1 labelled data :  0.0
reward :  (5, -1, False, '{"true_positive_rate": 0, "false_positive_rate": 3, "true_negative_rate": 0, "false_negative_rate": 0}')


# Exploring data and creating pytorch model

link : https://www.kaggle.com/dakshmiglani/pytorch-credit-card-fraud-prediction-99-8

In [0]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils

In [17]:
df = pd.read_csv('./dataset/creditcard.csv')
df.head(1) # give us a sneek preview of the dataset xD

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.5516,-0.617801,-0.99139,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0


In [0]:
X = df.iloc[:, :-1].values 
y = df.iloc[:, -1].values

sc = StandardScaler()
X = sc.fit_transform(X)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# Creating Deep Q-Network

## 1. Creating Experience Replay

In [0]:
from collections import deque

class Experience():
  def __init__(self, state, action, reward, next_state):
    self.state = state
    self.action = action
    self.reward = reward
    self.next_state = next_state

class ReplayMemory():
  def __init__(self, max_size=1000):
    self.buffer = deque(maxlen=max_size)
  
  def add(self, experience: Experience):
    self.buffer.append(experience)
  
  def sample(self, batch_size):
    idx = np.random.choice(np.arange(len(self.buffer)),
                           size=batch_size, 
                           replace=False)
    
    return [self.buffer[i] for i in idx]


## 2. Defining the Q-Network

In [0]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        # returning 2 q-values, one for every action
        self.fc5 = nn.Linear(24, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [145]:
net = DQN().double()
net

DQN(
  (fc1): Linear(in_features=30, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=18, bias=True)
  (fc3): Linear(in_features=18, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=24, bias=True)
  (fc5): Linear(in_features=24, out_features=2, bias=True)
)

In [0]:
X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 3. Define the Agent


In [0]:
from agent_SARSA import Agent

In [0]:
dqn_agent = Agent(nA=2)

# Training the model

## Optimize model using experience replay

In [0]:
train = data_utils.TensorDataset(X_train, Y_train)
train_loader = data_utils.DataLoader(train, batch_size=1, shuffle=True)

In [151]:
x = torch.tensor([[random.randrange(2)]], device=device, dtype=torch.long)
print(x)

tensor([[1]])


In [0]:
policy_net = DQN().double().to(device)

"""
 tensor([[ 0.0497, -0.2222],
        [ 0.0445, -0.2421],
        [ 0.0564, -0.2190],
        [ 0.0522, -0.2444],
        [ 0.0546, -0.2387],
        [ 0.0773, -0.2068],

"""

def get_epsilon_greedy_policy(state):
  if random.random()>0:
    print('Using DQN to get action')
    #with torch.no_grad():
    action = policy_net(state).max(1)[1].float()
  else:
    print('Selecting random action')
    action = torch.tensor([[random.randrange(2)]], device=device)
  
  return action
  


In [156]:
for i in range(200):
  env.reset()
  accumulated_reward = 0
  for state_idx, data in enumerate(train_loader, 0):
    inputs, labels = data
    action = get_epsilon_greedy_policy(inputs)
    optimizer.zero_grad()
    print('action : ',action)
    print('labels : ', labels)
    loss = criterion(action.double(), labels)  
    loss.backward()
    optimizer.step()
    print('Training loss : ',loss)


Using DQN to get action
action :  tensor([0.])
labels :  tensor([0.], dtype=torch.float64)


RuntimeError: ignored